In [1]:
import torch
import random
import numpy as np
import pandas as pd
from torch import nn
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score, accuracy_score
from torch.utils.data import Dataset, DataLoader
from transformers import (
    RobertaTokenizer,
    RobertaForSequenceClassification,
    AdamW,
    get_cosine_schedule_with_warmup
)
import torch.nn.functional as F
from tqdm import tqdm

# =========================================================
# 1. GENERAL CONFIG & SEEDING
# =========================================================

def seed_everything(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

seed_everything()

device = "cuda" if torch.cuda.is_available() else "cpu"

print("Using device:", device)

# =========================================================
# 2. LOAD DATASET
# =========================================================

df = pd.read_csv("emotion_master_200k_balanced.csv")

# Use only emotion as target (training first model)
texts = df["text"].tolist()
labels = df["emotion"].tolist()

label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)
num_labels = len(label_encoder.classes_)

df["label"] = labels

# =========================================================
# 3. GENERALIZED NLP AUGMENTATIONS
# =========================================================

def random_mask(text, p=0.15):
    """Randomly mask some tokens"""
    words = text.split()
    if len(words) < 4:
        return text
    for i in range(len(words)):
        if random.random() < p:
            words[i] = "<mask>"
    return " ".join(words)

def word_dropout(text, p=0.10):
    """Randomly drop some words"""
    words = text.split()
    new_words = []
    for w in words:
        if random.random() > p:
            new_words.append(w)
    return " ".join(new_words) if new_words else text

def random_swap(text):
    """Swap two words randomly"""
    words = text.split()
    if len(words) < 3:
        return text
    idx1, idx2 = random.sample(range(len(words)), 2)
    words[idx1], words[idx2] = words[idx2], words[idx1]
    return " ".join(words)

def augment(text):
    """Apply multiple augmentation layers"""
    if random.random() < 0.30:
        text = random_mask(text)
    if random.random() < 0.30:
        text = word_dropout(text)
    if random.random() < 0.20:
        text = random_swap(text)
    return text

# =========================================================
# 4. DATASET CLASS
# =========================================================

tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

class EmotionDataset(Dataset):
    def __init__(self, df, augment_data=False):
        self.df = df
        self.augment_data = augment_data

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        text = self.df.iloc[idx]["text"]
        label = self.df.iloc[idx]["label"]

        if self.augment_data:
            text = augment(text)

        enc = tokenizer(
            text,
            max_length=96,
            truncation=True,
            padding="max_length",
            return_tensors="pt"
        )

        return {
            "input_ids": enc["input_ids"].squeeze(),
            "attention_mask": enc["attention_mask"].squeeze(),
            "labels": torch.tensor(label, dtype=torch.long),
            "text_embedding": enc["input_ids"]  # used for mixup
        }

# =========================================================
# 5. MIXUP LOSS FOR NLP (Helps Generalization)
# =========================================================

def mixup(emb1, emb2, y1, y2, alpha=0.4):
    lam = np.random.beta(alpha, alpha)
    emb = lam * emb1 + (1 - lam) * emb2
    y = (lam, y1, y2)
    return emb, y

def mixup_loss(logits, y_mix):
    lam, y1, y2 = y_mix
    return lam * F.cross_entropy(logits, y1) + (1 - lam) * F.cross_entropy(logits, y2)

# =========================================================
# 6. TRAIN ONE FOLD
# =========================================================

def train_fold(train_df, val_df, fold=1):

    train_data = EmotionDataset(train_df, augment_data=True)
    val_data = EmotionDataset(val_df, augment_data=False)

    train_loader = DataLoader(train_data, batch_size=16, shuffle=True)
    val_loader = DataLoader(val_data, batch_size=32)

    model = RobertaForSequenceClassification.from_pretrained(
        "roberta-base",
        num_labels=num_labels
    )
    model.to(device)

    optimizer = AdamW(model.parameters(), lr=2e-5, weight_decay=0.01)

    total_steps = len(train_loader) * 4
    scheduler = get_cosine_schedule_with_warmup(
        optimizer,
        num_warmup_steps=int(0.1 * total_steps),
        num_training_steps=total_steps
    )

    best_f1 = 0
    patience_counter = 0

    scaler = torch.cuda.amp.GradScaler()

    print(f"\n======== Training FOLD {fold} ========\n")

    for epoch in range(4):
        model.train()
        total_loss = 0

        loop = tqdm(train_loader, desc=f"Epoch {epoch+1}")

        for batch in loop:
            optimizer.zero_grad()

            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            # MIXUP 50% of the time
            if random.random() < 0.50:
                idx2 = torch.randperm(input_ids.size(0))
                emb1 = input_ids.float()
                emb2 = input_ids[idx2].float()
                y1 = labels
                y2 = labels[idx2]
                emb_mix, y_mix = mixup(emb1, emb2, y1, y2)

                with torch.cuda.amp.autocast():
                    outputs = model(
                        input_ids=emb_mix.long(),
                        attention_mask=attention_mask,
                        labels=None
                    )
                    loss = mixup_loss(outputs.logits, y_mix)

            else:
                with torch.cuda.amp.autocast():
                    outputs = model(
                        input_ids=input_ids,
                        attention_mask=attention_mask,
                        labels=labels
                    )
                    # Label smoothing (epsilon = 0.1)
                    loss = F.cross_entropy(outputs.logits, labels, label_smoothing=0.1)

            scaler.scale(loss).backward()
            nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            scaler.step(optimizer)
            scaler.update()
            scheduler.step()

            total_loss += loss.item()

        # ================= VALIDATION =================
        model.eval()
        preds, actuals = [], []

        with torch.no_grad():
            for batch in val_loader:
                input_ids = batch["input_ids"].to(device)
                attention_mask = batch["attention_mask"].to(device)
                labels = batch["labels"].to(device)

                with torch.cuda.amp.autocast():
                    logits = model(
                        input_ids=input_ids,
                        attention_mask=attention_mask
                    ).logits

                preds.extend(torch.argmax(logits, dim=1).cpu().numpy())
                actuals.extend(labels.cpu().numpy())

        f1 = f1_score(actuals, preds, average="macro")
        acc = accuracy_score(actuals, preds)

        print(f"FOLD {fold} EPOCH {epoch+1} → F1: {f1:.4f} | Acc: {acc:.4f}")

        # EARLY STOPPING
        if f1 > best_f1:
            best_f1 = f1
            patience_counter = 0

            model.save_pretrained(f"generalized_emotion_model_fold{fold}")
            tokenizer.save_pretrained(f"generalized_emotion_model_fold{fold}")

            print("🔥 Best model updated!")

        else:
            patience_counter += 1

        if patience_counter >= 2:
            print("⛔ Early stopping triggered")
            break

    print(f"Best F1 for Fold {fold}: {best_f1:.4f}")


# =========================================================
# 7. K-FOLD TRAINING
# =========================================================

kf = KFold(n_splits=3, shuffle=True, random_state=42)

fold = 1
for train_idx, val_idx in kf.split(df):
    train_fold(df.iloc[train_idx], df.iloc[val_idx], fold=fold)
    fold += 1


c:\Users\sinth\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\cuda\__init__.py:61: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]
c:\Users\sinth\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

...

FOLD 3 EPOCH 2 → F1: 0.9313 | Acc: 0.9324
🔥 Best model updated!


Epoch 3:   0%|          | 0/8334 [00:00<?, ?it/s]C:\Users\sinth\AppData\Local\Temp\ipykernel_20044\2257095105.py:199: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Epoch 3:   0%|          | 1/8334 [00:00<32:07,  4.32it/s]C:\Users\sinth\AppData\Local\Temp\ipykernel_20044\2257095105.py:208: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Epoch 3: 100%|██████████| 8334/8334 [31:45<00:00,  4.37it/s]
C:\Users\sinth\AppData\Local\Temp\ipykernel_20044\2257095105.py:235: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


FOLD 3 EPOCH 3 → F1: 0.9685 | Acc: 0.9687
🔥 Best model updated!


Epoch 4:   0%|          | 0/8334 [00:00<?, ?it/s]C:\Users\sinth\AppData\Local\Temp\ipykernel_20044\2257095105.py:208: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Epoch 4:   0%|          | 2/8334 [00:00<33:28,  4.15it/s]C:\Users\sinth\AppData\Local\Temp\ipykernel_20044\2257095105.py:199: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Epoch 4: 100%|██████████| 8334/8334 [31:47<00:00,  4.37it/s]
C:\Users\sinth\AppData\Local\Temp\ipykernel_20044\2257095105.py:235: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


FOLD 3 EPOCH 4 → F1: 0.9765 | Acc: 0.9767
🔥 Best model updated!
Best F1 for Fold 3: 0.9765


In [6]:
from transformers import RobertaForSequenceClassification, RobertaTokenizer

# path where your best model was automatically saved earlier
SOURCE_DIR = "generalized_emotion_model_fold1"   # or fold2 or fold3
TARGET_DIR = "final_emotion_model"

# load saved model
model = RobertaForSequenceClassification.from_pretrained(SOURCE_DIR)
tokenizer = RobertaTokenizer.from_pretrained(SOURCE_DIR)

# save again wherever you want
model.save_pretrained(TARGET_DIR)
tokenizer.save_pretrained(TARGET_DIR)

print("Model saved to:", TARGET_DIR)


Model saved to: final_emotion_model


In [7]:
from transformers import RobertaForSequenceClassification

model_dir = "final_emotion_model"

# load safetensors
model = RobertaForSequenceClassification.from_pretrained(model_dir)

# save as pytorch_model.bin
model.save_pretrained(model_dir, safe_serialization=False)

print("Converted to pytorch_model.bin!")


Converted to pytorch_model.bin!


In [8]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Load your dataset
df = pd.read_csv("emotion_master_200k_balanced.csv")

# Fit label encoder on the "emotion" column
label_encoder = LabelEncoder()
label_encoder.fit(df["emotion"].tolist())

# Print labels
print("Number of labels:", len(label_encoder.classes_))
print("Emotion labels:")
for i, label in enumerate(label_encoder.classes_):
    print(f"{i}: {label}")


Number of labels: 18
Emotion labels:
0: anger
1: anxious
2: bored
3: calm
4: confused
5: depressed
6: excited
7: fear
8: frustrated
9: grateful
10: happy
11: lonely
12: motivated
13: relieved
14: sad
15: stress
16: surprised
17: tired


In [3]:
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification

# -------------------------------
# LOAD LABEL NAMES (important!)
# -------------------------------
id2label = {
    0: "anger",
    1: "anxious",
    2: "bored",
    3: "calm",
    4: "confused",
    5: "depressed",
    6: "excited",
    7: "fear",
    8: "frustrated",
    9: "grateful",
    10: "happy",
    11: "lonely",
    12: "motivated",
    13: "relieved",
    14: "sad",
    15: "stress",
    16: "surprised",
    17: "tired"
}

# -------------------------------
# LOAD MODEL FROM YOUR FOLDER
# -------------------------------
MODEL_DIR = "final_emotion_model"   # <-- change if needed

tokenizer = RobertaTokenizer.from_pretrained(MODEL_DIR)
model = RobertaForSequenceClassification.from_pretrained(MODEL_DIR)
model.eval()

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# -------------------------------
# PREDICTION FUNCTION
# -------------------------------
def predict_emotion(text):
    encoding = tokenizer(
        text,
        truncation=True,
        padding="max_length",
        max_length=96,
        return_tensors="pt"
    )

    input_ids = encoding["input_ids"].to(device)
    attention_mask = encoding["attention_mask"].to(device)

    with torch.no_grad():
        logits = model(input_ids=input_ids, attention_mask=attention_mask).logits

    pred_id = torch.argmax(logits, dim=1).item()
    emotion = id2label[pred_id]

    return emotion

# -------------------------------
# TEST
# -------------------------------
text = "I am feeling very tired and stressed today"
prediction = predict_emotion(text)
print("Predicted Emotion:", prediction)


Predicted Emotion: tired


In [15]:
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification

# ======================================================
# 1. Load your saved model (change folder if needed)
# ======================================================
MODEL_DIR = "generalized_emotion_model_fold1"  # change fold if needed

tokenizer = RobertaTokenizer.from_pretrained(MODEL_DIR)
model = RobertaForSequenceClassification.from_pretrained(MODEL_DIR)
model.eval()

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# ======================================================
# 2. Emotion Labels (same as training)
# ======================================================
id2label = {
    0: "anger",
    1: "anxious",
    2: "bored",
    3: "calm",
    4: "confused",
    5: "depressed",
    6: "excited",
    7: "fear",
    8: "frustrated",
    9: "grateful",
    10: "happy",
    11: "lonely",
    12: "motivated",
    13: "relieved",
    14: "sad",
    15: "stress",
    16: "surprised",
    17: "tired"
}

# ======================================================
# 3. Emotion → Suggestion Mapping
# ======================================================
suggestions = {
    "anger": "Take a walk or listen to calming music.",
    "anxious": "Try deep breathing or meditation. Nearby calm places recommended.",
    "bored": "Explore nearby cafes or entertainment spots.",
    "calm": "Everything looks good. Maybe enjoy a coffee nearby.",
    "confused": "Would you like help or guidance?",
    "depressed": "Consider talking to someone or visiting peaceful places.",
    "excited": "Great mood! Try celebration spots or restaurants.",
    "fear": "You may need safe locations like police station or trusted places.",
    "frustrated": "A short break at a café or walk can help.",
    "grateful": "Maybe send a thank-you message or celebrate!",
    "happy": "Enjoy! You can explore trending places nearby.",
    "lonely": "Try social spots, cafes, or call a friend.",
    "motivated": "Good time to start a task or visit a library/workspace.",
    "relieved": "Relax! Maybe enjoy a peaceful place.",
    "sad": "Try comforting places like a peaceful park or tea shop.",
    "stress": "Consider visiting a restaurant or taking short break.",
    "surprised": "Interesting! Would you like recommendations?",
    "tired": "You need rest. Suggesting coffee/tea shops."
}

# ======================================================
# 4. Classification + Suggestion Function
# ======================================================

def predict_emotion_and_suggestion(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(device)

    with torch.no_grad():
        logits = model(**inputs).logits
        pred_id = torch.argmax(logits, dim=1).item()

    emotion = id2label[pred_id]
    suggestion = suggestions[emotion]

    return emotion, suggestion

# ======================================================
# 5. TEST
# ======================================================

text = input("Enter your message: ")

emotion, suggestion = predict_emotion_and_suggestion(text)

print("\n============================")
print(f"Predicted Emotion: {emotion}")
print(f"Suggested Action : {suggestion}")
print("============================")



Predicted Emotion: surprised
Suggested Action : Interesting! Would you like recommendations?


In [4]:
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification

# ------------------------------
# 1. Load Saved Model & Tokenizer
# ------------------------------
MODEL_DIR = "final_emotion_model"   # change if needed

tokenizer = RobertaTokenizer.from_pretrained(MODEL_DIR)
model = RobertaForSequenceClassification.from_pretrained(MODEL_DIR)
model.eval()

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# ------------------------------
# 2. Emotion Label Mapping (18 classes)
# ------------------------------
id2label = {
    0: "anger",
    1: "anxious",
    2: "bored",
    3: "calm",
    4: "confused",
    5: "depressed",
    6: "excited",
    7: "fear",
    8: "frustrated",
    9: "grateful",
    10: "happy",
    11: "lonely",
    12: "motivated",
    13: "relieved",
    14: "sad",
    15: "stress",
    16: "surprised",
    17: "tired"
}

# ------------------------------
# 3. Prediction Function
# ------------------------------
def predict_emotion(text):
    enc = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        padding="max_length",
        max_length=96
    )

    input_ids = enc["input_ids"].to(device)
    attention_mask = enc["attention_mask"].to(device)

    with torch.no_grad():
        logits = model(input_ids, attention_mask=attention_mask).logits
        pred_id = torch.argmax(logits, dim=1).item()
        pred_label = id2label[pred_id]

    return pred_label

# ------------------------------
# 4. Test It
# ------------------------------
test_text = "I am feeling so stressed and worried today"
predicted_emotion = predict_emotion(test_text)

print("Predicted Emotion:", predicted_emotion)


Predicted Emotion: stress


In [5]:
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification
import numpy as np

# ==========================================
# 1. Load your trained model
# ==========================================
MODEL_PATH = "final_emotion_model"   # change if needed

device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = RobertaTokenizer.from_pretrained(MODEL_PATH)
model = RobertaForSequenceClassification.from_pretrained(MODEL_PATH)
model.to(device)
model.eval()

# ==========================================
# 2. Emotion label mapping (your dataset)
# ==========================================
label_map = {
    0: "anger",
    1: "anxious",
    2: "bored",
    3: "calm",
    4: "confused",
    5: "depressed",
    6: "excited",
    7: "fear",
    8: "frustrated",
    9: "grateful",
    10: "happy",
    11: "lonely",
    12: "motivated",
    13: "relieved",
    14: "sad",
    15: "stress",
    16: "surprised",
    17: "tired"
}

# ==========================================
# 3. Automatic place suggestions based ONLY on the model output
# ==========================================
emotion_to_place = {
    "anger": ["gym", "boxing club", "running track"],
    "anxious": ["meditation center", "calm park"],
    "bored": ["gaming cafe", "mall", "cinema"],
    "calm": ["library", "quiet cafe"],
    "confused": ["bookstore", "study cafe"],
    "depressed": ["park", "quiet tea shop"],
    "excited": ["tea shop", "street food", "hangout cafe"],
    "fear": ["public space", "crowded cafe"],
    "frustrated": ["coffee shop", "walkway"],
    "grateful": ["temple", "family restaurant"],
    "happy": ["restaurant", "ice cream shop"],
    "lonely": ["cafe", "community center"],
    "motivated": ["gym", "work cafe"],
    "relieved": ["park", "beach side"],
    "sad": ["quiet cafe", "tea shop"],
    "stress": ["spa", "calm tea shop"],
    "surprised": ["new restaurant", "unique cafe"],
    "tired": ["tea shop", "resting area"]
}

# ==========================================
# 4. Function: classify text and get place suggestions
# ==========================================
def suggest_place(user_text):
    enc = tokenizer(
        user_text,
        return_tensors="pt",
        truncation=True,
        padding=True,
        max_length=96
    )

    input_ids = enc["input_ids"].to(device)
    attention_mask = enc["attention_mask"].to(device)

    with torch.no_grad():
        logits = model(input_ids, attention_mask=attention_mask).logits

    pred_idx = torch.argmax(logits, dim=1).item()
    emotion = label_map[pred_idx]

    # suggestions come ONLY from model prediction
    places = emotion_to_place.get(emotion, ["general hangout place"])

    return emotion, places

# ==========================================
# 5. Example input
# ==========================================
user_input = "I am feeling very sad"

emotion, places = suggest_place(user_input)

print("Predicted Emotion:", emotion)
print("Suggested Places:", places)


Predicted Emotion: sad
Suggested Places: ['quiet cafe', 'tea shop']
